In [2]:
import cv2,os
import numpy as np
from keras.models import load_model
model =load_model('model2.h5')

In [20]:
img_size = 150
img_test=r'\test2.jpg'
img1 = cv2.imread(img_test)
gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
cv2.imwrite('gray.jpg',gray1)
resize = cv2.resize(gray1,(img_size,img_size))

data1 = np.array(resize)/255.0  #data values are normalized
#reshaping of data                                                
data1 = np.reshape(data1,(1,150,150,1))

yhat = model.predict(data1, verbose=0)
label = np.argmax(yhat,axis=1)[0]
if (label):
    print("mask is present")
else:
    print("no mask")

no mask


In [22]:
import cv2,os
import numpy as np
from keras.models import load_model
model =load_model('model2.h5')
img_size = 150
faceCascade=cv2.CascadeClassifier(r"\haar.xml")
video_capture = cv2.VideoCapture(0)  #starts the webcam
labels_dict = {0:'NO MASK',1:'MASK'}
color_dict  = { 0:(0,0,255),1:(0,255,0)}
while(True):
    ret,frame = video_capture.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,1.3,5)
    
    for x,y,w,h in faces:
        face_img = gray[y:y+w,x:x+h]
        resized = cv2.resize(face_img,(img_size,img_size))
        normalized = resized/255.0
        reshaped = np.reshape(normalized,(1,img_size,img_size,1))
        result = model.predict(reshaped)
        
        label = np.argmax(result,axis=1)[0]
        cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(frame,labels_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    cv2.imshow('Video',frame)
    key=cv2.waitKey(1)
    
    if(key==27):
        break;
        
cv2.destroyAllWindows()
video_capture.release()

In [23]:
import cv2,os
import numpy as np
from keras.models import load_model
import tkinter
from tkinter import messagebox
import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import time
from playsound import playsound


#root=tkinter.Tk
#root.withdraw
model=load_model("model2.h5")

img_size = 150
faceCascade=cv2.CascadeClassifier(r"\haar.xml")
video_capture = cv2.VideoCapture(0)  #starts the webcam
labels_dict = {0:'NO MASK',1:'MASK ON'}
color_dict  = { 0:(0,0,255),1:(0,255,0)}

#make sure to allow third party access on your email to use this function
SUBJECT="ACCESS DENIED"
TEXT="The current visitor is not wearing a mask, hence, access denied"
sender_email = "enter your email"
receiver_email = "enter senders email, can be same too"
password = "enter password of email id"

# Create a multipart message and set headers
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = SUBJECT


# Add body to email
message.attach(MIMEText(TEXT,TEXT))



while(True):
    ret,frame = video_capture.read()
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,1.3,5)
    
    for x,y,w,h in faces:
        face_img = gray[y:y+w,x:x+h]
        resized = cv2.resize(face_img,(img_size,img_size))
        normalized = resized/255.0
        reshaped = np.reshape(normalized,(1,img_size,img_size,1))
        result = model.predict(reshaped)
        
        label = np.argmax(result,axis=1)[0]
        cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(frame,labels_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        #time.sleep(5)
        if (label==0):
            #messagebox.showwarning("WARNING","ACCESS DENIED. PLEASE WEAR A FACE MASK")
            playsound(r'\Downloads\ad.wav')
            cv2.imwrite('image.png',frame) 
            filename="image.png"

            with open(filename, "rb") as attachment:
                part = MIMEBase("application", "octet-stream")
                part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header( "Content-Disposition",f"attachment; filename= {filename}",)
            message.attach(part)
            text = message.as_string()

            context = ssl.create_default_context()
            with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
                server.login(sender_email, password)
                server.sendmail(sender_email, receiver_email, text)
            os.remove('image.png')
        else:
            pass
            break
    cv2.imshow('Video',frame)  
    key=cv2.waitKey(1)
    
    if(key==27):
        break;

cv2.destroyAllWindows()
video_capture.release()
